* https://www.kaggle.com/sunmiyoon/one-hour-analysis-written-in-both-eng-and-kor/comments#290148 를 따라하기
* https://www.kaggle.com/primaryobjects/voicegender
* https://pandas.pydata.org/pandas-docs/stable/10min.html

In [94]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))

data를 읽은 후,
1. label로부터 gender(float64)를 만들고,
2. train set과 test set으로 나눈다.

In [150]:
_data = pd.read_csv('../input/voice.csv')
v_data = _data.reindex(columns=list(_data.columns) + ['gender'])
#v_data.dtypes
_len = len(v_data)
print(_len)
print(v_data.at[3167, 'label'])
for i in range(_len):
    if v_data.at[i, 'label'] == 'male':
        v_data.at[i, 'gender'] = 1.0
    else:
        v_data.at[i, 'gender'] = 0.0
print(v_data.at[3167, 'gender'])
v_data = v_data.drop(['label'], axis=1)
# shuffling
v_data = v_data.reindex(np.random.permutation(v_data.index))
print(len(v_data.columns), v_data.columns)
_split = _len - 1000
train = v_data[0:_split]
test = v_data[_split:-1]
print(len(train), len(test))

## Data scanning
데이터 스캔

---

* 분석 전에 날것의 데이터를 그대로 스캔하는 것이 중요합니다. 어떤 변수들이 들어있고, 각자의 데이터 타입은 무엇인지, 어떤 변수는 dummy로 사용하고 어떤 변수는 numerical 로 사용 할 것인지 염두해 두면서 데이터를 스캔합니다.

In [101]:
print(train.tail(2))
# print(test.head(2))
# print(v_data.tail(2))
# print(test.tail(2))
# v_data.size
# v_data.columns

## Summary Statistics
요약 통계

---

In [102]:
print(train.describe())

### Correlation matrix
* First I see the correlation matrix to see the rough relationships between features(variables). If there is strong relationship between independent variables (Xs on the right side), we could not trust the coefficient of them. It could be under-estimated or over-estimated than a real effect to dependent variable(Y on the left side).  
* 첫번째로, 상관관계 테이블을 이용하여 변수들간의 상관관계를 살펴봅니다. 만약 독립변수(Xs)들 간에 상관관계가 높다면 회귀분석의 결과(coefficient)를 신뢰하기가 힘들어 집니다. 예를 들면, A라는 변수와 B라는 두 독립변수의 상관관계가 매우 높다면 A의 계수를 온전히 A가 Y에 미치는 영향이라고 해석하기 어렵습니다. 

In [103]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
f, ax = plt.subplots(figsize=(12,9))
sns.heatmap(train.corr(), vmax=.8, square=True);

* 두 변수의 상관관계가 높은 것이 히트맵에서는 하얀 칸으로 표시됩니다. 상관관계가 매우 높은 페어는 다음과 같습니다.
 * meanfreq - median
 * meanfreq - Q25
 * meanfreq - Q75
 * meanfreq - mode
 * meanfreq - centroid
 * sd - IQR
 * sd - sfm
 * median - Q25
 * median - Q75
 * median - mode
 * median - centroid
 * centroid - median
 * Q25 - centroid
 * Q75 - centroid
 * skew - kurt
 * sp.ent - sfm
 * mode - centroid
 * meandom - maxdom
 * meandom - dfrange
 * maxdom - dfrange
 
 각 페어 당 label과 상관관계가 더 높은 변수 하나만 포함시킵니다.
* 특정 변수가 Y에 미치는 영향을 정확하게 설명하고 싶다면 이 과정이 필수적입니다. 하지만 단지 Y를 잘 예측하는 것이 모델의 목적이라면 굳이 이 단계에서 변수를 제거 할 필요는 없습니다.

In [104]:
def strongerRelationSalePrice(f1, f2):
    f1Corr = train.corr().loc[f1,'gender']
    f2Corr = train.corr().loc[f2,'gender']
#     print(f1Corr, f2Corr)
    return (f1, f2) if (f1Corr >= f2Corr) else (f2, f1)

def print_stronger(f1, f2):
    print('{} > {}'.format(strongerRelationSalePrice(f1, f2)[0], strongerRelationSalePrice(f1, f2)[1]))

In [51]:
print_stronger('meanfreq', 'median')
print_stronger('meanfreq', 'Q25')
print_stronger('meanfreq', 'Q75')
print_stronger('meanfreq', 'mode')
print_stronger('meanfreq', 'centroid')
print_stronger('sd', 'IQR')
print_stronger('sd', 'sfm')
print_stronger('median', 'Q25')
print_stronger('median', 'Q75')
print_stronger('median', 'mode')
print_stronger('median', 'centroid')
print_stronger('Q25', 'centroid')
print_stronger('Q75', 'centroid')
print_stronger('skew', 'kurt')
print_stronger('sp.ent', 'sfm')
print_stronger('mode', 'centroid')
print_stronger('meandom', 'maxdom')
print_stronger('meandom', 'dfrange')
print_stronger('maxdom', 'dfrange')
print_stronger('mode', 'Q75')

* Q75 > mode > meanfreq > centroid > median > Q25
* IQR > sd > sfm
* kurt > skew
* sp.ent > sfm
* meandom > dfrange > maxdom

In [105]:
train = train.drop(['mode', 'meanfreq', 'centroid', 'median', 'Q25', 'sd', 'sfm', 'skew', 'sfm', 'dfrange', 'maxdom'], axis=1)
test = test.drop(['mode', 'meanfreq', 'centroid', 'median', 'Q25', 'sd', 'sfm', 'skew', 'sfm', 'dfrange', 'maxdom'], axis=1)
print(len(train.columns), train.columns)

In [106]:
f, ax = plt.subplots(figsize=(12,9))
sns.heatmap(train.corr(), vmax=.8, square=True)

## Histogram and Scatter chart
히스토그램과 스캐터 차트

---

* 먼저, 가장 중요한 gender 데이터를 확인 해 봐야 합니다. gender에 음수는 없는지, 말도 안되게 큰 값은 없는지 체크하면서 데이터가 과연 활용할만한 데이터인지 살펴봐야 합니다. 물론 캐글은 깨끗한 데이터를 제공하니 이런 문제가 많지 않지만, 웹 크롤링을 얻은 데이터나 실생활 또는 업무에서 만날 수 있는 데이터들은 이처럼 깨끗하지 않습니다. 아웃라이어를 처리하는 방법에는 winsorization, truncation 등이 있습니다. 

In [107]:
%matplotlib inline
import matplotlib.pyplot as plt

# I think this graph is more elegant than pandas.hist()
# train['SalePrice'].hist(bins=100)
sns.distplot(train['gender'])

### Scatter chart
산점도, 스캐터 차트
* Y축은 모두 gender이고, 독립변수를 X축에 맞춰 모든 변수에 대해 산점도를 그렸습니다. 데이터 시각화는 보기에 멋있어 보이기 위해 하는 것만은 아닙니다. 이렇게 모든 독립변수들에 대해 산점도를 뿌리면 어떤 변수가 특히 종속변수과 연관이 있는지, 다시 말하면 얼마나 종속변수를 설명해 주는지 한 눈에 볼 수 있습니다. 예를 들어, OverallQual과 SalePrice가 양의 관계를 보이며 산점도가 우상향하는 형태를 보이는 것을 확인 할 수 있습니다. 데이터가 수평에 가깝게 뿌려져 있다면 그 변수는 gender 와 낮은 관계를 가진다고 해석할 수도 있습니다. 하지만 이 데이터셋의 경우에는 변수의 갯수도 많고, 변수들간 관계가 복잡하기 때문에 회귀분석을 하기 이전에 성급하게 결론을 내려서는 안 됩니다.

In [108]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 6, figsize=(15, 7), sharey=True)
for col, a in zip(train.columns, axes.flatten()):
    if col == 'gender':   
        a.set_title(col)
        a.scatter(df['gender'], df['gender'])
    else:
        df = train[['gender', col]].dropna()
        a.set_title(col)
        a.scatter(df[col], df['gender'])

* 이 data를 lab5-ex.ipynb에 적용해보자.

In [109]:
# Lab 5 Logistic Regression Classifier
import tensorflow as tf
import numpy as np
tf.set_random_seed(743)  # for reproducibility

# collect data
x_data = train.loc[:,['Q75','IQR','kurt','sp.ent']].values
y_data = train.loc[:,['gender']].values

In [132]:
print(x_data[0],y_data[0])
#len(x_data)
type(x_data)
#y_data

* build a model

In [146]:
X = tf.placeholder(tf.float32, shape=[None, 4])
Y = tf.placeholder(tf.float32, shape=[None, 1])
W = tf.Variable(tf.random_normal([4, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(cost)

predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

* train a model

In [147]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2000):
    cost_val, _ = sess.run([cost, optimizer], feed_dict={X: x_data, Y: y_data})
    if step % 10 == 0:
        print(step, cost_val)

x_test = test.loc[:,['Q75','IQR','kurt','sp.ent']].values
y_test = test.loc[:,['gender']].values
h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: x_test, Y: y_test})
print("Accuracy: ", a)
print(c[0:8], y_test[0:8])